Primero cargaremos todo las librerias que usaremos y tambien el archivo.csv de canciones

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate
from sklearn.neighbors import NearestNeighbors

songs= pd.read_csv('../spotify_songs.csv')
songs.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


Cargamos el csv de vector caracteristico de las canciones

In [4]:
features = pd.read_csv('../features.csv')
features.head()

,track_id,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature41,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50
0,047fCsbO4NdmwCBn8pcUXl,0.500593,0.329950,0.361185,0.303097,0.413862,0.339932,0.279063,0.364843,0.363516,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,05oFyvdztPdpwSwpoclD4c,0.322660,0.310444,0.471902,0.326134,0.450965,0.365918,0.381789,0.387558,0.321200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,03sno6MIkoVX5eC4pRKMzr,0.528890,0.314285,0.366243,0.485485,0.334010,0.388274,0.355649,0.543254,0.459913,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,05LCfd8s9n2teUcsTsBAnr,0.459056,0.373905,0.326302,0.329807,0.326330,0.317138,0.338123,0.374200,0.439815,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,044fQJmT2fZUYcA3BnaOL2,0.447421,0.433604,0.550555,0.413023,0.314888,0.420090,0.354317,0.477022,0.326651,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Verficamos que tenemos lo necesario par auna busqueda textual con pandas solo extraemos las columnas "track_name", "track_artist", "lyrics","track_id". Esto no es necesario, pero se implemento con KNN.

In [5]:
essential = songs[["track_name", "track_artist", "lyrics","track_id"]]
essential.head()

,track_name,track_artist,lyrics,track_id
0,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,0017A6SJgTbfQVU2EtsPNo
1,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",004s3t0ONYlzxII9PLgU6z
2,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",00chLpzhgVjxs1zKC9UScL
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,00cqd6ZsSkLZqGMlQCR0Zo
4,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,00emjlCv9azBN0fzuuyLqy


Limpieza de datos nulos y duplicados

In [6]:
essential = essential.dropna()
essential = essential.drop_duplicates()
essential.shape
essential.head()

,track_name,track_artist,lyrics,track_id
0,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,0017A6SJgTbfQVU2EtsPNo
1,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",004s3t0ONYlzxII9PLgU6z
2,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",00chLpzhgVjxs1zKC9UScL
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,00cqd6ZsSkLZqGMlQCR0Zo
4,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,00emjlCv9azBN0fzuuyLqy


Obtenemos el tfidvectorizer de nuestra dataframe

In [7]:
vectorizer = TfidfVectorizer()
vectorizer
X = vectorizer.fit_transform(essential["lyrics"])
X

<18194x102102 sparse matrix of type '<class 'numpy.float64'>'
	with 2120968 stored elements in Compressed Sparse Row format>

KNN secuencial de busqueda textual implementada

In [8]:
def KNN_secuencial(query, k):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X,query_vec).reshape((-1,))
    top_k_indices = results.argsort()[-k:][::-1]
    top_k_songs = essential.iloc[top_k_indices][["track_name", "track_artist"]]
    return top_k_songs
query = "I'm a rebel just for kicks, now"
k=5
print(KNN_secuencial(query, k))




                                 track_name       track_artist
15794                         Feel It Still  Portugal. The Man
17317                         Feel It Still  Portugal. The Man
8709   It's All on U (feat. Liam O'Donnell)           ILLENIUM
2006    It's All on U - T-Mass & LZRD Remix           ILLENIUM
12584           Rebel Rebel - 1997 Remaster        David Bowie


Tambien se implemento una variacion con heap

In [10]:
import heapq
def KNN_secuencial_heap(query, k):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X,query_vec).reshape((-1,))
    top_k_indices = heapq.nlargest(k, range(len(results)), results.take)
    top_k_songs = essential.iloc[top_k_indices][["track_name", "track_artist"]]
    return top_k_songs

print(KNN_secuencial_heap(query, k))

                                 track_name       track_artist
15794                         Feel It Still  Portugal. The Man
17317                         Feel It Still  Portugal. The Man
2006    It's All on U - T-Mass & LZRD Remix           ILLENIUM
8709   It's All on U (feat. Liam O'Donnell)           ILLENIUM
5462            Rebel Rebel - 2016 Remaster        David Bowie


Se implemento una busqueda textual con knn_range_search, no es necesario

In [11]:
radius = 0.2
def KNN_range_Search(query, radius):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X, query_vec).reshape((-1,))
    indices = np.where(results >= radius)[0]
    songs = essential.iloc[indices][["track_name", "track_artist"]]
    return songs

print(KNN_range_Search(query, radius))

                                 track_name       track_artist
2006    It's All on U - T-Mass & LZRD Remix           ILLENIUM
5462            Rebel Rebel - 2016 Remaster        David Bowie
8709   It's All on U (feat. Liam O'Donnell)           ILLENIUM
9937                              Pumped Up          Klingande
11489                               McQueen          TrifeDrew
12584           Rebel Rebel - 1997 Remaster        David Bowie
14385           Rebel Rebel - 1999 Remaster        David Bowie
15399                             Pumped Up          Klingande
15720           Rebel Rebel - 2014 Remaster        David Bowie
15786              Rebel Rebel - Remastered        David Bowie
15794                         Feel It Still  Portugal. The Man
16985                         Teenage Kicks     The Undertones
17317                         Feel It Still  Portugal. The Man
18306                       Pumped Up Kicks  Foster The People


In [12]:
first_value = songs.loc[0]
print(first_value)

track_id                                               0017A6SJgTbfQVU2EtsPNo
track_name                                                           Pangarap
track_artist                                                  Barbie's Cradle
lyrics                      Minsan pa Nang ako'y napalingon Hindi ko alam ...
track_popularity                                                           41
track_album_id                                         1srJQ0njEQgd8w4XSqI4JQ
track_album_name                                                         Trip
track_album_release_date                                           2001-01-01
playlist_name                                              Pinoy Classic Rock
playlist_id                                            37i9dQZF1DWYDQ8wBxd7xt
playlist_genre                                                           rock
playlist_subgenre                                                classic rock
danceability                                                    

Esta parte es lo que se pidio implementar, extraemos las mismas columnas importantes "track_name", "track_artist", "lyrics","track_id" para verificar si resulto con existe vemos las primeras filas con .head() de nuestra dataframe.

In [13]:
essential_2 = songs[['track_id', 'track_name', 'track_artist', 'lyrics']]
essential_2.head()

,track_id,track_name,track_artist,lyrics
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu..."
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U..."
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...


Limpieza de datos:

- **Eliminamos Datos nulos**

- **Eliminamos Datos duplicados**

Verificamos la cantidad de elementos se redujo o se mantuvo

In [14]:
essential_2 = essential_2.dropna()
essential_2 = essential_2.drop_duplicates()
essential_2.shape


(18194, 4)

Hacemos el merge entre nuestra dataframe extraida essential2 y nuestro csv con el vector caracteristico cargado anteriormente.


In [15]:
combined_data = pd.merge(essential_2, features, on='track_id')
combined_data.shape

(2058, 54)

Limipieza de datos luego del merge:

- **Eliminamos Datos nulos**

- **Eliminamos Datos duplicados**



In [16]:
combined_data = combined_data.dropna() #Elimino la tabla con valores nulos
combined_data = combined_data.drop_duplicates() #Elimino los duplicados

combined_data.shape

(2058, 54)

Ahora realizaremos la normalizacion del vector caracteristico para eso de la dataframe mezclada solo eligiremos columnas pertenecientes al vector caracteristico:
feature1 hasta feature50

In [17]:

# Seleccionar solo las columnas numéricas
features_to_normalize = combined_data.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
features_to_normalize.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature41,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50
0,0.186783,0.313960,0.470686,0.338401,0.381696,0.290222,0.379664,0.468223,0.387875,0.453376,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.258788,0.308280,0.480829,0.386945,0.486098,0.309217,0.296758,0.394858,0.398341,0.429846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.491523,0.528583,0.450975,0.435650,0.441585,0.468128,0.549045,0.538764,0.581056,0.591341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.276362,0.242934,0.303423,0.419220,0.300271,0.328006,0.264648,0.257973,0.197914,0.220803,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.419527,0.319837,0.335713,0.382775,0.594782,0.402256,0.335276,0.330342,0.355163,0.370175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Alineamos las filas con el conjunto de datos

In [18]:
features_to_normalize = features_to_normalize.loc[combined_data.index]
features_to_normalize



,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature41,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50
0,0.186783,0.313960,0.470686,0.338401,0.381696,0.290222,0.379664,0.468223,0.387875,0.453376,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.258788,0.308280,0.480829,0.386945,0.486098,0.309217,0.296758,0.394858,0.398341,0.429846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.491523,0.528583,0.450975,0.435650,0.441585,0.468128,0.549045,0.538764,0.581056,0.591341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.276362,0.242934,0.303423,0.419220,0.300271,0.328006,0.264648,0.257973,0.197914,0.220803,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.419527,0.319837,0.335713,0.382775,0.594782,0.402256,0.335276,0.330342,0.355163,0.370175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,0.428063,0.251753,0.248414,0.337981,0.297210,0.381987,0.303461,0.452193,0.584452,0.323860,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2054,0.553526,0.408627,0.502690,0.540689,0.459864,0.470807,0.447189,0.545790,0.509345,0.461318,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2055,0.441209,0.489536,0.374868,0.467322,0.526453,0.408775,0.434361,0.396884,0.479114,0.422111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2056,0.326506,0.316397,0.311442,0.463714,0.328688,0.379701,0.265608,0.301476,0.344572,0.385025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Normalizamos las características numéricas del vector caracteristico con scaler.

In [19]:
scaler = preprocessing.StandardScaler()
scaled_features = scaler.fit_transform(features_to_normalize)
scaled_features

array([[-2.37769054, -0.82825034,  0.84599127, ...,  0.        ,
         0.        ,  0.        ],
       [-1.5729117 , -0.89654608,  0.96450188, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.02828179,  1.75218043,  0.61569148, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.46594098,  1.28271455, -0.27352737, ...,  0.        ,
         0.        ,  0.        ],
       [-0.81605402, -0.79895623, -1.01459281, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.15813721,  3.3100872 , -0.4455235 , ...,  0.        ,
         0.        ,  0.        ]])

Combinamos las características normalizadas con las otras columnas en una nueva dataframe con los datos normalizados

In [20]:
normalized_combined_data = pd.concat([combined_data[['track_id', 'track_name', 'track_artist', 'lyrics']], pd.DataFrame(scaled_features, columns=features_to_normalize.columns)], axis=1)
normalized_combined_data.head()


,track_id,track_name,track_artist,lyrics,feature1,feature2,feature3,feature4,feature5,feature6,...,feature41,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,-2.377691,-0.828250,0.845991,-0.627082,-0.258815,-1.083091,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",-1.572912,-0.896546,0.964502,-0.075893,0.844988,-0.877021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",1.028282,1.752180,0.615691,0.477124,0.374368,0.846925,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,-1.376495,-1.682210,-1.108283,0.290573,-1.119679,-0.673185,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,0.223607,-0.757594,-0.731013,-0.123240,1.994046,0.132309,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Limpieza de datos a esta nueva dataframe:

- **Verificar si hay valores NaN en el DataFrame**

- **Mostrar las filas con al menos un valor NaN**

- **Mostrar el número total de filas con al menos un valor NaN**

In [21]:
nan_check = normalized_combined_data.isna().any(axis=1)

rows_with_nan = normalized_combined_data[nan_check]
print(f"Filas con al menos un valor NaN:\n{rows_with_nan}")

total_rows_with_nan = rows_with_nan.shape[0]
print(f"Número total de filas con al menos un valor NaN: {total_rows_with_nan}")


Filas con al menos un valor NaN:
Empty DataFrame
Columns: [track_id, track_name, track_artist, lyrics, feature1, feature2, feature3, feature4, feature5, feature6, feature7, feature8, feature9, feature10, feature11, feature12, feature13, feature14, feature15, feature16, feature17, feature18, feature19, feature20, feature21, feature22, feature23, feature24, feature25, feature26, feature27, feature28, feature29, feature30, feature31, feature32, feature33, feature34, feature35, feature36, feature37, feature38, feature39, feature40, feature41, feature42, feature43, feature44, feature45, feature46, feature47, feature48, feature49, feature50]
Index: []

[0 rows x 54 columns]
Número total de filas con al menos un valor NaN: 0


Verficamos nuestra dataframe limpia.

In [22]:
normalized_combined_data

,track_id,track_name,track_artist,lyrics,feature1,feature2,feature3,feature4,feature5,feature6,...,feature41,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,-2.377691,-0.828250,0.845991,-0.627082,-0.258815,-1.083091,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",-1.572912,-0.896546,0.964502,-0.075893,0.844988,-0.877021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",1.028282,1.752180,0.615691,0.477124,0.374368,0.846925,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,-1.376495,-1.682210,-1.108283,0.290573,-1.119679,-0.673185,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,0.223607,-0.757594,-0.731013,-0.123240,1.994046,0.132309,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,0WEgOlUCB9nHPsoD8Ae4Zz,The Voices,dreamEater,"Heavy on the ground, I fall Running into every...",0.319005,-1.576169,-1.750997,-0.631844,-1.152047,-0.087573,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2054,0wErDMQQMhwO1DTbE6kpkt,goodbye,updog,Verse 1: Can't cope with myself tonight I thin...,1.721266,0.309937,1.219924,1.669767,0.567620,0.875994,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2055,0WEv09ugUmm35vJ8mGDepn,Hoy,Morbo,NA Seguí recorriendo los caminos que siempre q...,0.465941,1.282715,-0.273527,0.836739,1.271643,0.203033,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2056,0WEVf4RhIrcr3S3ULBrt0T,LIKE I WOULD,ZAYN,Hey what's up? It's been a while Talking 'bout...,-0.816054,-0.798956,-1.014593,0.795771,-0.819243,-0.112374,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Esta parte parece ser innecesaria, pero se realiza y es que a partir del vector caracteristico se normalizara ahora con la columna lyrycs de la cancion para obetener un vector caracteristico mas exacto.

In [23]:
# Seleccionar las características numéricas normalizadas
numeric_features = normalized_combined_data.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
numeric_features

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature41,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50
0,-2.377691,-0.828250,0.845991,-0.627082,-0.258815,-1.083091,-0.103483,0.691857,-0.212519,0.368720,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.572912,-0.896546,0.964502,-0.075893,0.844988,-0.877021,-1.032394,-0.121985,-0.096271,0.108210,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.028282,1.752180,0.615691,0.477124,0.374368,0.846925,1.794336,1.474374,1.933313,1.896175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.376495,-1.682210,-1.108283,0.290573,-1.119679,-0.673185,-1.392172,-1.640456,-2.322596,-2.206168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.223607,-0.757594,-0.731013,-0.123240,1.994046,0.132309,-0.600822,-0.837666,-0.575879,-0.552423,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,0.319005,-1.576169,-1.750997,-0.631844,-1.152047,-0.087573,-0.957289,0.514027,1.971036,-1.065189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2054,1.721266,0.309937,1.219924,1.669767,0.567620,0.875994,0.653098,1.552306,1.136753,0.456645,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2055,0.465941,1.282715,-0.273527,0.836739,1.271643,0.203033,0.509360,-0.099510,0.800950,0.022578,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2056,-0.816054,-0.798956,-1.014593,0.795771,-0.819243,-0.112374,-1.381414,-1.157875,-0.693523,-0.388021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
# Crear un nuevo vectorizador para las letras
text_vectorizer = TfidfVectorizer()
text_features = text_vectorizer.fit_transform(normalized_combined_data["lyrics"])
text_features

<2058x30557 sparse matrix of type '<class 'numpy.float64'>'
	with 241865 stored elements in Compressed Sparse Row format>

In [25]:
scaler = preprocessing.StandardScaler()
scaled_numeric_features = scaler.fit_transform(numeric_features)
scaled_numeric_features

array([[-2.37769054, -0.82825034,  0.84599127, ...,  0.        ,
         0.        ,  0.        ],
       [-1.5729117 , -0.89654608,  0.96450188, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.02828179,  1.75218043,  0.61569148, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.46594098,  1.28271455, -0.27352737, ...,  0.        ,
         0.        ,  0.        ],
       [-0.81605402, -0.79895623, -1.01459281, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.15813721,  3.3100872 , -0.4455235 , ...,  0.        ,
         0.        ,  0.        ]])

In [26]:
# Combinar las características normalizadas con las otras columnas
combined_features = np.hstack((scaled_numeric_features, text_features.toarray()))
combined_features

array([[-2.37769054, -0.82825034,  0.84599127, ...,  0.        ,
         0.        ,  0.        ],
       [-1.5729117 , -0.89654608,  0.96450188, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.02828179,  1.75218043,  0.61569148, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.46594098,  1.28271455, -0.27352737, ...,  0.        ,
         0.        ,  0.        ],
       [-0.81605402, -0.79895623, -1.01459281, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.15813721,  3.3100872 , -0.4455235 , ...,  0.        ,
         0.        ,  0.        ]])

Obtenemos el mismo vector caracteristico no hay diferencia.

In [27]:
normalized_combined_data

,track_id,track_name,track_artist,lyrics,feature1,feature2,feature3,feature4,feature5,feature6,...,feature41,feature42,feature43,feature44,feature45,feature46,feature47,feature48,feature49,feature50
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,-2.377691,-0.828250,0.845991,-0.627082,-0.258815,-1.083091,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",-1.572912,-0.896546,0.964502,-0.075893,0.844988,-0.877021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",1.028282,1.752180,0.615691,0.477124,0.374368,0.846925,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,-1.376495,-1.682210,-1.108283,0.290573,-1.119679,-0.673185,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,0.223607,-0.757594,-0.731013,-0.123240,1.994046,0.132309,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,0WEgOlUCB9nHPsoD8Ae4Zz,The Voices,dreamEater,"Heavy on the ground, I fall Running into every...",0.319005,-1.576169,-1.750997,-0.631844,-1.152047,-0.087573,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2054,0wErDMQQMhwO1DTbE6kpkt,goodbye,updog,Verse 1: Can't cope with myself tonight I thin...,1.721266,0.309937,1.219924,1.669767,0.567620,0.875994,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2055,0WEv09ugUmm35vJ8mGDepn,Hoy,Morbo,NA Seguí recorriendo los caminos que siempre q...,0.465941,1.282715,-0.273527,0.836739,1.271643,0.203033,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2056,0WEVf4RhIrcr3S3ULBrt0T,LIKE I WOULD,ZAYN,Hey what's up? It's been a while Talking 'bout...,-0.816054,-0.798956,-1.014593,0.795771,-0.819243,-0.112374,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Esta parte es necesaria para la query de modo que crearemos un vector caracteristico para esta consulta, necesariamente la misma cantidad de caracteristicas que de las canciones.

In [69]:
def extract_and_normalize_features(audio_path, normalized_combined_data):
    y, sr = librosa.load(audio_path)

    # Extraer características de audio (aquí puedes elegir las características que deseas)
    features = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=50)  # Por ejemplo, MFCC con 50 coeficientes

    # Asegurarse de que las características tengan el mismo tamaño que las del DataFrame normalized_combined_data
    max_length = normalized_combined_data.shape[1] - 4  # Restamos 4 para excluir las columnas 'track_id', 'track_name', 'track_artist', 'lyrics'

    # Ajustar el tamaño de las características (rellenando con ceros si es necesario)
    if features.shape[1] < max_length:
        pad_width = max_length - features.shape[1]
        features = np.pad(features, pad_width=((0, 0), (0, pad_width)))

    # Crear un DataFrame con las características del audio
    audio_features = pd.DataFrame(features.T, columns=[f'feature_{i}' for i in range(1, 51)])

    # Convertir el DataFrame de características de audio a un array NumPy
    audio_features_array = audio_features.values

    # Normalizar el vector de características de audio
    scaled_audio_features = scaler.transform(audio_features_array)

    # Convertir el array NumPy a un DataFrame
    scaled_audio_features = pd.DataFrame(scaled_audio_features, columns=[f'feature_{i}' for i in range(1, 51)])

    
    return scaled_audio_features

Consulta 2

In [78]:
audio_path="../query2.wav"
scaled_audio_features2=extract_and_normalize_features(audio_path, normalized_combined_data)
scaled_audio_features2

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50
0,-435.298279,6.181631e+01,-5.518703e+01,-5.283036e+01,1.006228e+01,1.593364e+01,-1.183124e+01,-1.184160e+01,-6.563836e+00,-2.022074e+01,...,1.890657,-4.152495,-1.600627,0.637568,-1.987611,-3.078083,-2.041412,-2.035029,-0.254819,2.687773
1,-432.778809,6.362511e+01,-5.540108e+01,-5.211942e+01,1.208072e+01,1.729493e+01,-1.110637e+01,-1.027745e+01,-4.780220e+00,-2.076568e+01,...,0.820626,-3.511550,-1.561490,-0.095371,-1.630478,-2.258565,-2.139597,-1.680360,1.806483,4.813932
2,-491.277191,2.719347e+01,-2.790563e+01,-2.843440e+01,7.896958e+00,1.984876e+01,3.292487e+00,-6.046800e+00,-4.562005e+00,-1.079455e+01,...,3.447924,2.457991,-0.137030,-0.535500,0.287070,-1.140657,-3.710199,-3.333452,-0.282164,1.150508
3,-527.434998,4.778519e+00,-4.767743e+00,-6.015786e+00,1.653944e+00,6.482426e+00,1.431564e+00,-4.117432e+00,-6.697977e+00,-1.559728e+00,...,1.164773,1.956649,1.553669,-0.069646,-0.426701,-0.195562,-0.888655,-0.314521,0.271944,0.544531
4,-526.483032,4.786145e+00,6.505978e-01,-4.101000e+00,-2.521240e+00,1.598745e-01,-3.807217e+00,-1.062407e+00,-4.440026e+00,1.331767e+00,...,0.477625,-0.533394,0.580311,0.762936,0.940194,-0.111534,-2.233724,-0.093682,-0.414478,-0.065711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,-536.169312,1.360368e-02,1.134204e-02,7.845156e-03,3.495392e-03,-1.234266e-03,-5.830289e-03,-9.791218e-03,-1.268307e-02,-1.420349e-02,...,0.011119,0.007547,0.003152,-0.001585,-0.006150,-0.010047,-0.012851,-0.014255,-0.014112,-0.012430
99,-536.179443,1.035777e-16,1.657243e-16,4.894044e-16,9.839878e-17,3.038278e-16,1.381035e-16,3.625218e-17,3.210907e-16,3.871215e-16,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100,-536.179443,1.035777e-16,1.657243e-16,4.894044e-16,9.839878e-17,3.038278e-16,1.381035e-16,3.625218e-17,3.210907e-16,3.871215e-16,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
101,-536.014221,1.633314e-01,8.302107e-03,-1.184806e-01,-1.518423e-01,-1.164014e-01,-7.196701e-02,-3.849478e-02,1.024084e-02,8.787709e-02,...,0.139791,0.124893,0.066877,-0.032570,-0.129096,-0.163100,-0.117421,-0.033911,0.034084,0.074851


Consulta 1

In [77]:
audio_path="../query.wav"
scaled_audio_features=extract_and_normalize_features(audio_path, normalized_combined_data)

scaled_audio_features

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50
0,-568.804565,1.035777e-16,1.657243e-16,4.894044e-16,9.839878e-17,3.038278e-16,1.381035e-16,3.625218e-17,3.210907e-16,3.871215e-16,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-568.804565,1.035777e-16,1.657243e-16,4.894044e-16,9.839878e-17,3.038278e-16,1.381035e-16,3.625218e-17,3.210907e-16,3.871215e-16,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-568.804565,1.035777e-16,1.657243e-16,4.894044e-16,9.839878e-17,3.038278e-16,1.381035e-16,3.625218e-17,3.210907e-16,3.871215e-16,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-568.804565,1.035777e-16,1.657243e-16,4.894044e-16,9.839878e-17,3.038278e-16,1.381035e-16,3.625218e-17,3.210907e-16,3.871215e-16,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-568.804565,1.035777e-16,1.657243e-16,4.894044e-16,9.839878e-17,3.038278e-16,1.381035e-16,3.625218e-17,3.210907e-16,3.871215e-16,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,-568.804565,1.035777e-16,1.657243e-16,4.894044e-16,9.839878e-17,3.038278e-16,1.381035e-16,3.625218e-17,3.210907e-16,3.871215e-16,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,-568.804565,1.035777e-16,1.657243e-16,4.894044e-16,9.839878e-17,3.038278e-16,1.381035e-16,3.625218e-17,3.210907e-16,3.871215e-16,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,-568.804565,1.035777e-16,1.657243e-16,4.894044e-16,9.839878e-17,3.038278e-16,1.381035e-16,3.625218e-17,3.210907e-16,3.871215e-16,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,-568.804565,1.035777e-16,1.657243e-16,4.894044e-16,9.839878e-17,3.038278e-16,1.381035e-16,3.625218e-17,3.210907e-16,3.871215e-16,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,-568.804565,1.035777e-16,1.657243e-16,4.894044e-16,9.839878e-17,3.038278e-16,1.381035e-16,3.625218e-17,3.210907e-16,3.871215e-16,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Consulta 3

In [79]:
audio_path="../query3.wav"
scaled_audio_features3=extract_and_normalize_features(audio_path, normalized_combined_data)

scaled_audio_features3

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50
0,-452.724274,16.924856,2.595886,0.188916,9.110559,14.837120,9.008940,-1.973846,-6.261281,-1.994055,...,1.702420,-0.657482,-4.234949,-4.738449,-2.213763,0.267299,-0.167193,-2.402987,-1.888680,1.545091
1,-410.642242,37.878498,-12.398136,7.635675,25.848841,17.341076,18.234722,10.302134,-8.106704,-4.588545,...,-0.089016,-3.761619,-4.470116,-4.636983,-4.037546,-0.541969,1.499311,-0.312359,-3.566075,-3.098645
2,-382.672241,53.071121,-20.443975,9.030219,28.894205,14.160756,19.827663,14.348463,-7.230200,-7.883796,...,-0.945809,-4.176041,-1.323466,-1.838003,-4.715570,-2.603651,-0.357314,1.138924,1.156595,-0.311161
3,-369.403137,54.236336,-33.565346,5.604999,29.356882,13.356710,18.183220,12.924965,-0.766858,-2.856497,...,-2.135226,-5.825880,-1.965517,0.317736,0.718074,-0.646404,-2.323295,2.034595,3.187683,-1.233164
4,-319.774872,49.737312,-72.241028,18.592773,38.982628,16.524385,10.810252,8.151426,18.440136,-1.164516,...,-4.205997,-7.724287,-2.218155,2.965809,3.062781,-2.084713,-0.701308,2.215660,0.209100,1.663980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,-456.339050,18.189573,13.573991,10.442705,9.125338,6.622579,1.514627,-3.427662,-4.695999,-2.712117,...,0.649452,0.020208,-0.494807,0.260423,1.309992,1.105334,-0.230457,-1.079945,-0.524492,0.532870
121,-452.356323,20.595327,8.629278,-0.078671,0.502267,7.542910,12.368683,8.731424,-1.401681,-9.806206,...,-2.062413,-2.359034,-0.349051,1.852670,2.200044,0.536542,-1.409368,-1.689159,0.117682,2.575477
122,-441.799774,31.053013,8.969999,-8.212903,-8.861092,3.981013,16.216305,15.558817,1.994557,-13.084056,...,-2.663283,-0.764747,2.907993,5.754568,5.604769,2.531068,-1.194563,-2.934605,-1.791005,0.729393
123,-441.696930,31.396715,9.171690,-9.674070,-13.238663,-2.395521,10.987670,14.438184,4.874936,-9.837373,...,-2.520688,-0.373323,3.164268,5.942469,5.781009,2.362501,-2.209871,-4.850890,-3.912940,-0.603882


EL modelo KNN search se implementa con ayudade sklearn que nos proporciona la facilidad con los modelos de dos arrays se logra obtener los k vecinos mas cercanos

In [82]:
from sklearn.neighbors import NearestNeighbors

def knn_search(query_audio_features, reference_features, k=5, metric='cosine'):
    # Inicializar el modelo KNN
    knn_model = NearestNeighbors(n_neighbors=k, metric=metric)

    # Extraer las características numéricas para el modelo KNN, 4 primeras columnas del data frame
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
    knn_model.fit(reference_features_numeric)

    # Convertir query_audio_features a un array NumPy
    query_features_numeric = query_audio_features.to_numpy()

    # Utilizar el modelo para encontrar los k vecinos más cercanos para la query del audio
    distances, indices = knn_model.kneighbors(query_features_numeric)

    # Obtener las k canciones más cercanas utilizando los índices
    nearest_songs = reference_features.iloc[indices[0], :4]  # Obtener solo las primeras 4 columnas

    return nearest_songs


Las querys 1 y 3 son casi las mismas diferente diccion y volumen, mismos resultados.

In [83]:
k_nearest_songs = knn_search(scaled_audio_features, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
1652,0RHjhISKIx9SzyDDZQNk6v,Quiero Cantar,Abel Pintos,No imaginar un mundo sin fronteras Nos amarra ...
421,0aZPhHp1DVNVS5HAWzpUsM,Everybody Wants To Rule The World - Extended V...,Tears For Fears,Welcome to your life There's no turning back E...
2027,0W2pKKNHnfZZgcAxDPW0hy,Sofia,Alvaro Soler,NA Sueño cuando era pequeño Sin preocupación e...
1646,0ReSY5XOCFXFRooRS7hPxk,God Only Knows (GATTÜSO Remix),for KING & COUNTRY,Wide awake while the world is sound asleepin' ...
1310,0mtUvL8w3vQiMm93AfnRpO,Welcome to My Nightmare,Alice Cooper,Welcome to my nightmare I think you're gonna l...


In [84]:
k_nearest_songs = knn_search(scaled_audio_features2, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
1652,0RHjhISKIx9SzyDDZQNk6v,Quiero Cantar,Abel Pintos,No imaginar un mundo sin fronteras Nos amarra ...
935,0HsE2zUH1uZg7Bnsjf51TM,Work It Out,Knox Hamilton,I know what you're doing when you're falling a...
1355,0Ni4MKSGgvn0IGdnF4upti,Back To You,Madison Mars,"I ran from home, lost all I know To change my ..."
1283,0mJDqliT6vmi7i9kRoSAZ3,We Won't Go Home,The Colourist,Maybe they'll find us some day Maybe they won'...
356,0a4vQ58TiGl8Z77EEAvQCx,"Uncertain Smile - Original 7"" Version 1982",The The,Peeling the skin back from my eyes I felt surp...


In [85]:
k_nearest_songs = knn_search(scaled_audio_features3, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
1652,0RHjhISKIx9SzyDDZQNk6v,Quiero Cantar,Abel Pintos,No imaginar un mundo sin fronteras Nos amarra ...
2027,0W2pKKNHnfZZgcAxDPW0hy,Sofia,Alvaro Soler,NA Sueño cuando era pequeño Sin preocupación e...
1646,0ReSY5XOCFXFRooRS7hPxk,God Only Knows (GATTÜSO Remix),for KING & COUNTRY,Wide awake while the world is sound asleepin' ...
421,0aZPhHp1DVNVS5HAWzpUsM,Everybody Wants To Rule The World - Extended V...,Tears For Fears,Welcome to your life There's no turning back E...
1310,0mtUvL8w3vQiMm93AfnRpO,Welcome to My Nightmare,Alice Cooper,Welcome to my nightmare I think you're gonna l...


El range search utilizamos el modelo KNN y obtenemos resultados interesantes.

In [147]:
def range_search(query_features=pd.DataFrame(), reference_features=pd.DataFrame(), radius=int()):
    # Creamos el modelo KNN
    knn_model = NearestNeighbors(radius=radius, metric='euclidean')

    # Solo trabajaremos con el vector caracteristicos de las canciones
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)

    # Entranamos al modelo KNN con el vector caracteristico de las canciones
    knn_model.fit(reference_features_numeric)

    # Convertimos el dataframe de la consulta a un array numpy
    query_features_numeric = query_features.to_numpy()

    # Calculamos las distancias
    distances, _ = knn_model.kneighbors(query_features_numeric)

    # Extraemos las distancias y que sean menores o iguales al radio para obtener las canciones mas cercanas.
    song_data = []
    for distance_array in distances:
        if all(distance_array <= radius):
            # Extraemos el primer elemento del array
            distance = distance_array[0]
            song_data.append({
                'track_id': reference_features['track_id'][int(distance)],
                'track_name': reference_features['track_name'][int(distance)],
                'track_artist': reference_features['track_artist'][int(distance)],
            })

    return song_data

Query 1 normalizada mala diccion y bajo volumen. 

In [148]:
radius = 260
indices_within_radius = range_search(scaled_audio_features, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,07eET3PAzpSz5CLE4wGgN5,Inocente,Myke Towers
1,07CQ0ux2CwbaVN05sfG1zN,Sleepy Eyes,Elohim
2,07fqonJcksfEG2Fav9SDA2,Make Believe,Kenny Lattimore
3,07idyjb93qYavu1s29HtBi,Drop In the Ocean,OMI
4,07idyjb93qYavu1s29HtBi,Drop In the Ocean,OMI


Query 2 diccion aceptable y volumen aceptable

In [149]:
radius = 265
indices_within_radius = range_search(scaled_audio_features2, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,07KHJvlYBeQVqrmifTEqEp,Dream On,Aerosmith
1,07jFllpAwipdNXstXHFtHk,Mine,Mark Mendy


La query 3 es mas clara mejor diccion y pronunciacion mejor resultado que la query 1, demostrando que si la consulta es una que no tiene una buena diccion y volumen bajo, obtenemos malos resultados.

In [157]:
radius = 240
indices_within_radius = range_search(scaled_audio_features3, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,06mzeP9GQNSHWd1Nh2C5FM,MIDDLE OF THE NIGHT,Monsta X
1,06HUJRcD0t95ogCb2pMQJF,Golden Years - M-Phazes Remix,Ruel
2,06H5AH5KtVWbAPNuOG8biW,No Te Quiero Perder,Sabino
3,06cX5CpHPVUYnEJ2UupXXp,Popeye,MC Igu
4,06HUJRcD0t95ogCb2pMQJF,Golden Years - M-Phazes Remix,Ruel
5,06WD650Zt5MQ55kFMP2f3r,Fire Fly,Childish Gambino
6,06YVA5Ajbk0Kv4ddFSEuBM,You Really Got Me - 2004 Remaster,Van Halen
7,06Z7Hy2LUzbcSFmWzwkun1,With You - Frank Walker Remix,Tyler Shaw
8,06qKEP3ITnVPmH2ZW1zYvn,Shotgun - Quintino Remix,Yellow Claw
9,06ZIzHhJFWf0mDyEylIPf0,Peachy Keen,L Devine
